In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Load the Excel workbook into a DataFrame
df = pd.read_excel('stock_data_with_log.xlsx')

# Define the list of stock tickers
tickers = ['NVDA', 'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'BRK-B', 'JNJ', 'V']

# Calculate log returns for each stock if not already calculated
log_returns = {}
for ticker in tickers:
    if df[f'{ticker}_Log_Return'].isnull().all():
        df[f'{ticker}_Log_Return'] = np.log(df[f'{ticker}_Adj_Close'] / df[f'{ticker}_Adj_Close'].shift(1))
    log_returns[ticker] = df[f'{ticker}_Log_Return']

# Calculate log returns for the market index (SPY)
if df['SPY_Log_Return'].isnull().all():
    df['SPY_Log_Return'] = np.log(df['SPY_Adj_Close'] / df['SPY_Adj_Close'].shift(1))
log_returns['SPY'] = df['SPY_Log_Return']

# Drop the first row with NaN values due to shift operation
df.dropna(inplace=True)

# Calculate excess returns (subtracting the risk-free rate)
for ticker in tickers:
    df[f'{ticker}_Excess_Return'] = df[f'{ticker}_Log_Return'] - df['1M_TBill_Rate'] / 12
df['SPY_Excess_Return'] = df['SPY_Log_Return'] - df['1M_TBill_Rate'] / 12

# Initialize dictionaries to store alpha and beta values
alpha = {}
beta = {}

# Perform linear regression to obtain alpha and beta for each stock
for ticker in tickers:
    X = df['SPY_Excess_Return']
    Y = df[f'{ticker}_Excess_Return']
    X = sm.add_constant(X)  # Add a constant term for the intercept
    model = sm.OLS(Y, X).fit()
    alpha[ticker] = model.params['const']
    beta[ticker] = model.params['SPY_Excess_Return']

# Display the alpha and beta values
for ticker in tickers:
    print(f'{ticker}: Alpha = {alpha[ticker]}, Beta = {beta[ticker]}')

NVDA: Alpha = 0.020662156699603418, Beta = 1.776697133592898
AAPL: Alpha = 0.009573468996352006, Beta = 1.1942539513785706
MSFT: Alpha = 0.00960983626236518, Beta = 0.8809099533150359
GOOGL: Alpha = 0.002101569511902276, Beta = 1.0266017458275114
AMZN: Alpha = 0.0006648477817428752, Beta = 1.1774411380379646
META: Alpha = 0.0014881684449506928, Beta = 1.187115517803623
TSLA: Alpha = 0.014811992754853712, Beta = 1.80867564514114
BRK-B: Alpha = 0.0006290464983159949, Beta = 0.8540765085959384
JNJ: Alpha = -0.0027800333502016614, Beta = 0.5261673003825258
V: Alpha = 0.0022644041878425595, Beta = 0.8959661398160824


In [3]:
# Create a DataFrame for alpha and beta values
alpha_beta_df = pd.DataFrame({
    'Ticker': tickers,
    'Alpha': [alpha[ticker] for ticker in tickers],
    'Beta': [beta[ticker] for ticker in tickers]
})

# Load the existing Excel workbook and add the alpha and beta values to a new sheet
with pd.ExcelWriter('stock_data_with_log.xlsx', engine='openpyxl', mode='a') as writer:
    alpha_beta_df.to_excel(writer, sheet_name='Alpha_Beta', index=False)

# Display the alpha and beta values
print(alpha_beta_df)

  Ticker     Alpha      Beta
0   NVDA  0.020662  1.776697
1   AAPL  0.009573  1.194254
2   MSFT  0.009610  0.880910
3  GOOGL  0.002102  1.026602
4   AMZN  0.000665  1.177441
5   META  0.001488  1.187116
6   TSLA  0.014812  1.808676
7  BRK-B  0.000629  0.854077
8    JNJ -0.002780  0.526167
9      V  0.002264  0.895966
